In [1]:
import polars as pl
import os

# Base cleaning pipeline

1. `seendate` division: date, time, date_cdmx, time_cdmx
2. Strip news titles
3. Lowercase titles
4. Remove duplicates based on `oficial_url`

In [2]:
def format_dates(df: pl.DataFrame):
    return (
        df.with_columns(
            pl.col('seendate').str.strptime(pl.Datetime, "%Y%m%dT%H%M%SZ")
        )
        .with_columns(
            pl.col('seendate').dt.date().alias('date'),
            pl.col('seendate').dt.time().alias('time')
        )
        .with_columns(
            pl.col('seendate').dt.convert_time_zone('America/Mexico_City').alias('seendate_cdmx')
        )
        .with_columns(
            pl.col('seendate_cdmx').dt.date().alias('date_cdmx'),
            pl.col('seendate_cdmx').dt.time().alias('time_cdmx')
        )
    )

In [3]:
def strip_titles(df: pl.DataFrame):
    return (
        df.with_columns(
            pl.col("title").str.strip_chars()
        )
    )

In [4]:
def remove_duplicates(df: pl.DataFrame, date_col='seendate'):
    
    df_sorted = df.sort(date_col)
    deduped = df_sorted.unique(subset=['oficial_url'], keep="first")
    deduped = deduped.sort(date_col) 

    return deduped

In [5]:
def main():
    countries = ['mexico', 'us', 'canada', 'brasil', 'china']

    for country in countries:
        path = f'data/{country}/gdelt_raw.csv'

        df = pl.read_csv(path)

        print(f'#### {country.upper()} ####')

        ### Dates

        print(f'>> Formating dates <<')
        df_dates = format_dates(df)    
        print('>> Dates added successfully <<\n')

        ### Duplicated

        print(f'>> Removing duplicates <<')
        total_before = df_dates.height
        
        df_duplicates = remove_duplicates(df_dates)
        
        total_after = df_duplicates.height
        removed = total_before - total_after
        print(f'Total rows before cleaning: {total_before} — After: {total_after} — Removed: {removed}')
        
        print('>> Duplicates removed successfully <<\n')

        ### Titles
        
        print(f'>> Strip titles <<')
        df_titles = strip_titles(df_duplicates)
        print('>> Titles strip successfully <<\n')

        ### Saving
        
        output_path = f"data/base_clean/base_clean_{country}.csv"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        df_titles.write_csv(output_path)
        print(f"File saved successfully at: {output_path}\n")

In [6]:
main()

#### MEXICO ####
>> Formating dates <<
>> Dates added successfully <<

>> Removing duplicates <<
Total rows before cleaning: 105804 — After: 89362 — Removed: 16442
>> Duplicates removed successfully <<

>> Strip titles <<
>> Titles strip successfully <<

File saved successfully at: data/base_clean/base_clean_mexico.csv

#### US ####
>> Formating dates <<
>> Dates added successfully <<

>> Removing duplicates <<
Total rows before cleaning: 242353 — After: 196794 — Removed: 45559
>> Duplicates removed successfully <<

>> Strip titles <<
>> Titles strip successfully <<

File saved successfully at: data/base_clean/base_clean_us.csv

#### CANADA ####
>> Formating dates <<
>> Dates added successfully <<

>> Removing duplicates <<
Total rows before cleaning: 38444 — After: 29774 — Removed: 8670
>> Duplicates removed successfully <<

>> Strip titles <<
>> Titles strip successfully <<

File saved successfully at: data/base_clean/base_clean_canada.csv

#### BRASIL ####
>> Formating dates <<
>> D

# Cleaning steps for US data

1. Delete all domains from MX that are in US data, since they're already included in Mexico dataset
2. Delete [elceo.com, gob.mx, latinus.us] domains
3. For now, change the sourcecountry of China to United States, since i have no way to know if its indeed from china

In [15]:
df_us = pl.read_csv('data/base_clean/base_clean_us.csv')

print(df_us.shape)

(107432, 13)


In [16]:
# Eliminar los que tengan como país 'Mexico'
df_us = df_us.remove(pl.col('sourcecountry') == 'Mexico')

In [17]:
# A eliminar lo de elceo.com, gob.mx, latinus.us
to_delete = ['elceo.com', 'gob.mx', 'latinus.us']

df_us = df_us.remove(pl.col('domain').is_in(to_delete))

In [18]:
df_us.group_by("sourcecountry").agg(pl.len().alias("n"))

sourcecountry,n
str,u32
"""United States""",100081
"""China""",7351


In [19]:
# Por el momento, cambiar los nombres de china a eeuu 
df_us = df_us.with_columns(
    pl.when(pl.col("sourcecountry") == "China")
    .then(pl.lit("United States"))
    .otherwise(pl.col("sourcecountry"))
    .alias("sourcecountry")
)

df_us.head(1)

url,oficial_url,title,seendate,domain,language,sourcecountry,date,time,seendate_cdmx,date_cdmx,time_cdmx,country
str,str,str,str,str,str,str,str,str,str,str,str,str
"""https://api.gdeltproject.org/a…","""https://seekingalpha.com/artic…","""DBC : Why Commodities Will Lik…","""2020-01-01T01:00:00.000000""","""seekingalpha.com""","""English""","""United States""","""2020-01-01""","""01:00:00.000000000""","""2019-12-31T19:00:00.000000-060…","""2019-12-31""","""19:00:00.000000000""","""United States"""


In [20]:
df_us.group_by("language").agg(pl.len().alias("n"))

language,n
str,u32
"""Japanese""",4
"""English""",107041
"""Chinese""",14
"""Indonesian""",1
"""German""",28
"""Portuguese""",4
"""Arabic""",3
"""Spanish""",334
"""French""",3


In [21]:
df_us.shape

(107432, 13)

In [22]:
output_path = f"data/base_clean/base_clean_us.csv"
df_us.write_csv(output_path)
print(f"File saved successfully at: {output_path}\n")

File saved successfully at: data/base_clean/base_clean_us.csv

